In [21]:
import numpy as np

from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import Flatten, Dense
from keras.utils import to_categorical

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train =  x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [3]:
x_train.shape

(60000, 28, 28)

In [53]:
model = Sequential()
model.add(Flatten(input_shape=(28, 28)))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [54]:
model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 256)               200960    
_________________________________________________________________
dense_11 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_12 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                650       
Total params: 242,762
Trainable params: 242,762
Non-trainable params: 0
_________________________________________________________________


In [56]:
model.fit(x_train, y_train, 
          batch_size=64, epochs=10,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 8s 140us/step - loss: 0.2365 - acc: 0.9299 - val_loss: 0.1097 - val_acc: 0.9666
Epoch 2/10
60000/60000 [==============================] - 9s 149us/step - loss: 0.0908 - acc: 0.9722 - val_loss: 0.0936 - val_acc: 0.9720
Epoch 3/10
60000/60000 [==============================] - 7s 123us/step - loss: 0.0607 - acc: 0.9807 - val_loss: 0.0813 - val_acc: 0.9760
Epoch 4/10
60000/60000 [==============================] - 8s 136us/step - loss: 0.0464 - acc: 0.9856 - val_loss: 0.0710 - val_acc: 0.9789
Epoch 5/10
60000/60000 [==============================] - 7s 124us/step - loss: 0.0371 - acc: 0.9878 - val_loss: 0.0810 - val_acc: 0.9766
Epoch 6/10
60000/60000 [==============================] - 7s 113us/step - loss: 0.0276 - acc: 0.9911 - val_loss: 0.0755 - val_acc: 0.9793
Epoch 7/10
60000/60000 [==============================] - 7s 109us/step - loss: 0.0274 - acc: 0.9907 - val_loss: 0.0720 - 

In [57]:
model.evaluate(x_test, y_test, verbose=0)

[0.084290636213625841, 0.9788]

In [73]:
target = model.layers[1].get_weights()
u, s, v_t = np.linalg.svd(target[0], full_matrices=False)

for k in range(8):
    tmp_s = s.copy()
    tmp_s[2 ** k:] = 0
    new_weight = u @ np.diag(tmp_s) @ v_t
    
    model.layers[1].set_weights([new_weight, target[1]])
    
    print(model.evaluate(x_test, y_test, verbose=0))
model.layers[1].set_weights(target)

[4.6312937911987309, 0.20399999999999999]
[8.0829713439941404, 0.17230000000000001]
[7.0796395072937015, 0.25600000000000001]
[4.0804842388153073, 0.54959999999999998]
[1.0896742362964897, 0.80810000000000004]
[0.14903200184771559, 0.95979999999999999]
[0.089170128039208069, 0.97750000000000004]
[0.086126177126480616, 0.9778]


In [76]:
for k in range(8):
    new_model = Sequential()
    new_model.add(Flatten(input_shape=(28, 28)))
    new_model.add(Dense(2 ** k))
    new_model.add(Dense(256, activation='relu'))
    new_model.add(Dense(128, activation='relu'))
    new_model.add(Dense(64, activation='relu'))
    new_model.add(Dense(10, activation='softmax'))
    
    new_model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])
    
    new_model.fit(x_train, y_train, 
                  batch_size=64, epochs=10,
                  validation_data=(x_test, y_test), verbose=0)
    
    print(new_model.evaluate(x_test, y_test, verbose=0))

[1.3606835624694824, 0.4909]
[0.79150962805747982, 0.75060000000000004]
[0.36113099954724309, 0.88690000000000002]
[0.15023983775712549, 0.95369999999999999]
[0.10398878726357652, 0.97260000000000002]
[0.086782847858476453, 0.97819999999999996]
[0.093802640231544498, 0.97589999999999999]
[0.1304598247610731, 0.96450000000000002]
